In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from config_datasets import config_datasets
from cargar_dataset import cargar_dataset

for nombre, cfg in config_datasets.items():
    print(f"\n🔍 Analizando dataset: {nombre.upper()}")
    try:
        X, y_bin = cargar_dataset(
            path=cfg["path"],
            clase_minoria=cfg["clase_minoria"],
            col_features=cfg["col_features"],
            col_target=cfg["col_target"],
            sep=cfg["sep"],
            header=cfg["header"]
        )

        # Cargar target original (sin binarizar)
        df = pd.read_csv(cfg["path"], sep=cfg["sep"], header=cfg["header"])
        y_original = df[cfg["col_target"]]

        # Contar clases originales
        conteo = y_original.value_counts()
        clase_min_real = conteo.idxmin()
        total = conteo.sum()
        proporcion = (conteo / total * 100).round(2)

        # Mostrar
        print("🎯 Valores únicos del target:", list(conteo.index))
        print("📊 Distribución de clases:")
        for clase, count in conteo.items():
            print(f"   - {clase}: {count} ({proporcion[clase]}%)")
        print(f"✅ Clase minoritaria real: {clase_min_real}")
        print(f"⚠️ Clase configurada como minoritaria: {cfg['clase_minoria']}")

        if clase_min_real != cfg["clase_minoria"]:
            print("🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗")

    except Exception as e:
        print(f"❌ Error al analizar {nombre}: {e}")


In [ ]:
import pandas as pd
df = pd.read_csv("breast+cancer+wisconsin+original.csv", sep=",", header=None)
print(df.head())
print(df.columns)
